<a href="https://colab.research.google.com/github/dorirozen/AI_Tasks/blob/main/NlpTask3_Dori_Shlomi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title pips & imports
!pip install -q beautifulsoup4 transformers sumy tensorflow keras torch datasets
import nltk,time,spacy,csv,requests
from bs4 import BeautifulSoup
import transformers
from transformers import GPT2Tokenizer, GPT2Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from pandas import unique
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
import re

# Ensure you have the necessary NLTK data files
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
#@title BeautifulSoup part - We changed the URLs of the corpus source due to a large number of duplicate sentences that made the data very poor quality before we even started coding.
import requests
from bs4 import BeautifulSoup

class WikiScraper:
    def __init__(self):
        self.domain = "https://en.wikipedia.org/wiki/"
        self.sentences = []

    def scrape_wikipedia(self, subject):
        print(f"🌐 Scraping Wikipedia for '{subject}'...")
        url = f'{self.domain}{subject.replace(" ", "_")}'
        self.sentences = []
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check if the request was successful
        except requests.RequestException as e:
            print(f"Error fetching the Wikipedia page: {e}")
            return

        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find('div', {'id': 'mw-content-text'})
        if content:
            for element in content.find_all(['p', 'li']):
                text = element.get_text().strip()
                if text:  # Only append non-empty strings
                    self.sentences.append(text)

        if not self.sentences:
            print(f"No content found for {subject}")

    def save_to_file(self, filename):
        print(f"\n💾 Saving data to file...")
        with open(f'{filename}.txt', 'w', encoding='utf-8') as file:
            for sentence in self.sentences:
                file.write(sentence + '\n')
        print(f"Data saved to {filename}.txt")

    def load_from_file(self, filename):
        try:
            print(f"\n📂 Loading data from file...")
            with open(f'{filename}.txt', 'r', encoding='utf-8') as file:
                self.sentences = file.readlines()
            self.sentences = [sentence.strip() for sentence in self.sentences]
            print(f"Data loaded from {filename}.txt")
        except FileNotFoundError:
            print(f"File {filename}.txt not found.")

    def get_sentences(self):
        return self.sentences

    def print_stats(self):
        print(f"📊 Scraping Statistics:")
        print(f"   Total sentences: {len(self.sentences)}")
        print("\n📜 First 10 sentences (truncated to 100 characters):")
        for i, sentence in enumerate(self.sentences[:10], 1):
            print(f"   {i}. {sentence[:100]}...")

In [ ]:
#@title Our new corpus data
scraper = WikiScraper()

# Scrape Wikipedia
subject = "Python_(programming_language)"

scraper.scrape_wikipedia(subject)

filename = "python_corpus"
scraper.save_to_file(filename)
scraper.load_from_file(filename)
sentences = scraper.get_sentences()
print(f"✅ Scraping complete!")
scraper.print_stats()

CORPUS_PATH = f'/content/{filename}.txt'


🌐 Scraping Wikipedia for 'Python_(programming_language)'...

💾 Saving data to file...
Data saved to python_corpus.txt

📂 Loading data from file...
Data loaded from python_corpus.txt
✅ Scraping complete!
📊 Scraping Statistics:
   Total sentences: 955

📜 First 10 sentences (truncated to 100 characters):
   1. Python Programming at Wikibooks...
   2. Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code ...
   3. Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, inclu...
   4. Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming lan...
   5. Python consistently ranks as one of the most popular programming languages, and has gained widesprea...
   6. Python was invented in the late 1980s[42] by Guido van Rossum at Centrum Wiskunde & Informatica (CWI...
   7. Python 2.0 was released on 16 October 2000, with many major new features such as list comprehensions

In [ ]:
#@title TextPreprocessor

class TextPreprocessor:
    def __init__(self, options):
        self.options = options
        self.lemmatizer = WordNetLemmatizer()
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))

    def preprocess(self, text):
        if 'lowercase' in self.options:
            text = self._lowercase(text)
        if 'normalize' in self.options:
            text = self._normalize(text)
        if 'tokenize_sentence' in self.options:
            text = self._tokenize_sentence(text)
        if 'tokenize_word' in self.options:
            text = self._tokenize_word(text)
        if 'regex_tokenize' in self.options:
            text = self._regex_tokenize(text)
        if 'whitespace_tokenize' in self.options:
            text = self._whitespace_tokenize(text)
        if 'remove_stopwords' in self.options:
            text = self._remove_stopwords(text)
        if 'lemmatize' in self.options:
            text = self._lemmatize(text)
        if 'stem' in self.options:
            text = self._stem(text)
        return text

    def _lowercase(self, text):
        return text.lower()

    def _normalize(self, text):
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def _tokenize_sentence(self, text):
        return sent_tokenize(text)

    def _tokenize_word(self, text):
        return word_tokenize(text)

    def _regex_tokenize(self, text):
        tokenizer = RegexpTokenizer(r'\w+')
        return tokenizer.tokenize(text)

    def _whitespace_tokenize(self, text):
        return text.split()

    def _remove_stopwords(self, text):
        if isinstance(text, list):
            return [word for word in text if word.lower() not in self.stop_words]
        else:
            words = word_tokenize(text)
            return ' '.join([word for word in words if word.lower() not in self.stop_words])

    def _lemmatize(self, text):
        if isinstance(text, list):
            return [self.lemmatizer.lemmatize(word) for word in text]
        else:
            words = word_tokenize(text)
            return ' '.join([self.lemmatizer.lemmatize(word) for word in words])

    def _stem(self, text):
        if isinstance(text, list):
            return [self.stemmer.stem(word) for word in text]
        else:
            words = word_tokenize(text)
            return ' '.join([self.stemmer.stem(word) for word in words])

In [18]:
options = [
    'lowercase',
    'normalize',
    'tokenize_word',
    'remove_stopwords',
    'lemmatize'
]

preprocessor = TextPreprocessor(options)

print(f"sentences size: {len(sentences)}")
print(sentences[:5])

processed_sentences = [preprocessor.preprocess(sentence) for sentence in sentences]

print(processed_sentences)


sentences size: 955
['Python Programming at Wikibooks', 'Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.[33]', 'Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.[34][35]', 'Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python\xa00.9.0.[36] Python\xa02.0 was released in 2000. Python\xa03.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python\xa02.7.18, released in 2020, was the last release of Python\xa02.[37]', 'Python consistently ranks as one of the most popular programming languages, and has gained widesp

# RNN model and predicting the probability of the next word in a sentence.

In [25]:
#@title TextSequenceModel
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SimpleRNN, Dense

class TextSequenceModel:
    def __init__(self, sentences, max_sentences=200, embedding_dim=32, rnn_units=16, epochs=50, rnn_type='LSTM'):
        self.sentences = sentences[:max_sentences]
        self.embedding_dim = embedding_dim
        self.rnn_units = rnn_units
        self.epochs = epochs
        self.rnn_type = rnn_type
        self.tokenizer = Tokenizer()
        self.model = None
        self.max_seq_length = None
        self.input_sequences = None
        self.target_words = None
        self._prepare_data()
        self._build_model()

    def _prepare_data(self):
        self.tokenizer.fit_on_texts(self.sentences)
        sequences = self.tokenizer.texts_to_sequences(self.sentences)

        input_sequences = []
        target_words = []
        for seq in sequences:
            for i in range(1, len(seq)):
                input_sequences.append(seq[:i])
                target_words.append(seq[i])

        self.max_seq_length = max([len(seq) for seq in input_sequences])
        self.input_sequences = pad_sequences(input_sequences, maxlen=self.max_seq_length)

        self.input_sequences = np.array(self.input_sequences)
        self.target_words = np.array(target_words)

    def _build_model(self):

        self.model = Sequential()
        self.model.add(Embedding(input_dim=len(self.tokenizer.word_index) + 1, output_dim=self.embedding_dim, input_length=self.max_seq_length))
        if self.rnn_type == 'LSTM':
            self.model.add(LSTM(self.rnn_units))
        elif self.rnn_type == 'SimpleRNN':
            self.model.add(SimpleRNN(self.rnn_units))
        else:
            raise ValueError("rnn_type must be either 'LSTM' or 'SimpleRNN'")
        self.model.add(Dense(len(self.tokenizer.word_index) + 1, activation='softmax'))

        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    def train(self):

        history = self.model.fit(self.input_sequences, self.target_words, epochs=self.epochs, verbose=0)
        return history

    def predict_next_word(self, input_text):

        input_sequence = self.tokenizer.texts_to_sequences([input_text])[0]
        input_sequence = pad_sequences([input_sequence], maxlen=self.max_seq_length)
        predicted_probabilities = self.model.predict(input_sequence)[0]


        next_word_index = np.argmax(predicted_probabilities)

        next_word = self.tokenizer.index_word[next_word_index]

        probability = predicted_probabilities[next_word_index]

        return next_word, probability

    def predict_word_probability(self, input_text, next_word):

        input_sequence = self.tokenizer.texts_to_sequences([input_text])[0]
        input_sequence = pad_sequences([input_sequence], maxlen=self.max_seq_length)
        predicted_probabilities = self.model.predict(input_sequence)[0]

        next_word_index = self.tokenizer.word_index.get(next_word)

        if next_word_index is None:
            return 0.0


        return predicted_probabilities[next_word_index]


In [36]:
#@title training the models LSTM and RNN
print("🤖 Text Sequence Model Training\n")
print("=" * 80)
print("📊 LSTM Model")
print("=" * 80)
sentences_slice = processed_sentences[240:400]
lstm_model = TextSequenceModel(sentences_slice, rnn_type='LSTM')
lstm_history = lstm_model.train()

print("=" * 80)
print("📈 SimpleRNN Model")
print("=" * 80)
sentences_slice = processed_sentences[240:400]

rnn_model = TextSequenceModel(sentences_slice, rnn_type='SimpleRNN')
rnn_history = rnn_model.train()

print("✅ Training complete!")

#@title Comparing the two
print("=" * 80)
print("🔍 Model Comparison")
print("=" * 80)

print(f"\nLSTM Model:")
print(f"   Loss: {lstm_history.history['loss'][-1]:.4f}")
print(f"   Accuracy: {lstm_history.history['accuracy'][-1]:.4f}")

print(f"\nSimpleRNN Model:")
print(f"   Loss: {rnn_history.history['loss'][-1]:.4f}")
print(f"   Accuracy: {rnn_history.history['accuracy'][-1]:.4f}")

print("\n" + "=" * 80)
print("✅ Analysis complete!")

🤖 Text Sequence Model Training

📊 LSTM Model
📈 SimpleRNN Model
✅ Training complete!
🔍 Model Comparison

LSTM Model:
   Loss: 3.6149
   Accuracy: 0.2335

SimpleRNN Model:
   Loss: 2.6831
   Accuracy: 0.3886

✅ Analysis complete!


In [38]:
#@title print_model_predictions
from textwrap import fill
def print_model_predictions(model, model_name, input_sequence, specific_word):
    print("🤖 Text Sequence Model Predictions\n")
    print("=" * 80)
    print(f"📊 {model_name} Model")
    print("=" * 80)

    next_word, probability = model.predict_next_word(input_sequence)
    print("\n🔮 Prediction:")
    print(f"   Input: \"{input_sequence}\"")
    print(f"   Next word: \"{next_word}\"")
    print(f"   Probability: {probability:.4f}")

    specific_prob = model.predict_word_probability(input_sequence, specific_word)
    print(f"\n🎯 Probability of specific word:")
    print(f"   Word: \"{specific_word}\"")
    print(f"   Probability: {specific_prob:.4f}")

    print("\n" + "-" * 80 + "\n")

    if model_name == "SimpleRNN":
        print("✅ Predictions complete!")


input_sequence = "I love"
specific_word = "programming"
print_model_predictions(lstm_model, "LSTM", input_sequence, specific_word)
print("=" * 80)
print_model_predictions(rnn_model, "SimpleRNN", input_sequence, specific_word)

🤖 Text Sequence Model Predictions

📊 LSTM Model
1/1 [==============================] - 0s 106ms/step

🔮 Prediction:
   Input: "I love"
   Next word: "software"
   Probability: 0.0571
1/1 [==============================] - 0s 104ms/step

🎯 Probability of specific word:
   Word: "programming"
   Probability: 0.0060

--------------------------------------------------------------------------------

🤖 Text Sequence Model Predictions

📊 SimpleRNN Model
1/1 [==============================] - 0s 38ms/step

🔮 Prediction:
   Input: "I love"
   Next word: "python"
   Probability: 0.0673
1/1 [==============================] - 0s 34ms/step

🎯 Probability of specific word:
   Word: "programming"
   Probability: 0.0059

--------------------------------------------------------------------------------

✅ Predictions complete!


In [ ]:
#@title KLSummarizer
import nltk
from nltk.tokenize import word_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.kl import KLSummarizer
from textwrap import fill

nltk.download('punkt', quiet=True)

summarizer = KLSummarizer()

print("📚 Sentence Summaries:\n")

for i, sentence in enumerate(sentences[1:11], start=1):
    parser = PlaintextParser.from_string(sentence, Tokenizer("english"))
    summary = summarizer(parser.document, 1)

    print(f"{'=' * 80}")
    print(f"📌 Sentence {i}")
    print(f"{'=' * 80}\n")

    print("📜 Original:")
    print(fill(sentence, width=80, initial_indent="   ", subsequent_indent="   "))

    print("\n💡 Summary:")
    print(fill(str(summary[0]), width=80, initial_indent="   ", subsequent_indent="   "))

    print(f"\n{'-' * 80}\n")

print("✅ Summarization complete!")

📚 Sentence Summaries:

📌 Sentence 1

📜 Original:
   Python is a high-level, general-purpose programming language. Its design
   philosophy emphasizes code readability with the use of significant
   indentation.[33]

💡 Summary:
   Python is a high-level, general-purpose programming language.

--------------------------------------------------------------------------------

📌 Sentence 2

📜 Original:
   Python is dynamically typed and garbage-collected. It supports multiple
   programming paradigms, including structured (particularly procedural),
   object-oriented and functional programming. It is often described as a
   "batteries included" language due to its comprehensive standard
   library.[34][35]

💡 Summary:
   It supports multiple programming paradigms, including structured
   (particularly procedural), object-oriented and functional programming.

--------------------------------------------------------------------------------

📌 Sentence 3

📜 Original:
   Guido van Rossum began 

In [ ]:
#@title Load pre-trained model and tokenizer
import os
import random
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch

print("📚 Loading pre-trained model and tokenizer...")
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

📚 Loading pre-trained model and tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
#@title Using the pre-trained model and his tokenizer for fine tune
def load_dataset(file_path, tokenizer):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128)
    return dataset

print("🔍 Preparing dataset...")
if not os.path.exists(CORPUS_PATH):
    raise FileNotFoundError(f"Corpus file not found: {CORPUS_PATH}")
train_dataset = load_dataset(CORPUS_PATH, tokenizer)

print("🛠️ Setting up training configuration...")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("🚀 Starting fine-tuning process...")
trainer.train()

print("💾 Saving the fine-tuned model...")
trainer.save_model()
print("✅ Fine-tuning complete. Model saved.")


🔍 Preparing dataset...
🛠️ Setting up training configuration...
🚀 Starting fine-tuning process...


Step,Training Loss


💾 Saving the fine-tuned model...
✅ Fine-tuning complete. Model saved.


In [24]:
def generate_text(prompt, max_length=100):
    if not prompt.strip():
        return "Error: Empty prompt provided."

    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print("\n🤖 Text Generation from Partial Sentences")
print("=" * 80)

scraper.load_from_file(filename)
corpus = scraper.get_sentences()

# Filter out empty sentences
corpus = [sentence for sentence in corpus if sentence.strip()]

if not corpus:
    print("❌ Error: No valid sentences found in the corpus.")
else:
    partial_sentences = random.sample(corpus, min(5, len(corpus)))
    partial_sentences = [' '.join(sentence.split()[:max(1, len(sentence.split())//2)]) for sentence in partial_sentences]

    for i, partial in enumerate(partial_sentences, 1):
        print(f"\n📌 Sample {i}")
        print("=" * 80)

        print("🔍 Partial sentence:")
        print(f"   {partial}")

        completion = generate_text(partial)

        print(f"\n✨ Completion:")
        print(f"   {completion}")

        print("-" * 80)

print("\n✅ Text generation complete!")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Text Generation from Partial Sentences

📂 Loading data from file...
Data loaded from python_corpus.txt

📌 Sample 1
🔍 Partial sentence:
   Python provides a round function for rounding a float to the nearest integer. For tie-breaking, Python 3 uses round


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✨ Completion:
   Python provides a round function for rounding a float to the nearest integer. For tie-breaking, Python 3 uses round to find the smallest integer in the range from zero to infinity.
^ The "t" in a decimal is a unit of the number from which the decimal point is assigned. When rounding, "^" is an integer, and "e" the "E" a "number". The difference between the two numbers is the rounding error in each case, as in which case the
--------------------------------------------------------------------------------

📌 Sample 2
🔍 Partial sentence:
   Machine


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✨ Completion:
   Machine in Python
Jasmine, the first open-source computer programming language, has now been released and has a large community, including many in the Python community. Jasmin (pronounced "jas", from the original). JASM is a community of software developers, software engineers, engineers and security specialists who build and maintain the software they use. It is also the home of Jaspi, a Java-like language and Web-based data science platform.
The J
--------------------------------------------------------------------------------

📌 Sample 3
🔍 Partial sentence:
   ^ Esterbrook, Charles. "Acknowledgements". cobra-language.com. Cobra Language. Archived from


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✨ Completion:
   ^ Esterbrook, Charles. "Acknowledgements". cobra-language.com. Cobra Language. Archived from the original on 9 June 2020. Retrieved 5 May 2020.[9]
^ "An Introduction to Python". Python.org. Python Software Foundation. CPAN.
[10] "PEP 318 - Introduction and Python 3.0.3". peps.python.io. ppa.pepsi.opensource.pl. The python.cpython provides
--------------------------------------------------------------------------------

📌 Sample 4
🔍 Partial sentence:
   The if statement, which conditionally executes a block of


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✨ Completion:
   The if statement, which conditionally executes a block of code, e.g. if a function is called with a value of a certain value.
^ back to top
As a result of the above, a class is written as a variable named value, and a reference is made to it. The first part of an object is a list of values, and the third one is the name of that object. An object with an internal reference name can be used to create a new object (
--------------------------------------------------------------------------------

📌 Sample 5
🔍 Partial sentence:
   Python's

✨ Completion:
   Python's C compiler was inspired by the Python programming language, which was used in many other languages, including Java, Python, and C#.
The original syntax was borrowed from the original Python,[2] but was replaced with a more concise syntax.[3]
An important change was to the underlying program:
Python was a "pure" language,[4][5] and the syntax had a large influence on the development of other programs.[6] It als

In [39]:
#@title SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer
from textwrap import fill

sid = SentimentIntensityAnalyzer()

def get_sentiment_emoji(score):
    if score > 60:
        return "😃"
    elif score < 40:
        return "😔"
    else:
        return "😐"

for i, sentence in enumerate(sentences[1:11], start=1):
    print("🤖 Sentiment Analysis")
    print("=" * 80)
    sentiment_scores = sid.polarity_scores(sentence)

    positive_sentiment = sentiment_scores['pos'] * 100
    negative_sentiment = sentiment_scores['neg'] * 100
    neutral_sentiment = sentiment_scores['neu'] * 100

    print(f"\n{'=' * 80}")
    print(f"📌 Sentence {i}")
    print(f"{'=' * 80}")

    print("\n📜 Original:")
    print(fill(sentence, width=80, initial_indent="   ", subsequent_indent="   "))

    print("\n📊 Sentiment Analysis Statistics:")
    print(f"   {get_sentiment_emoji(positive_sentiment)} Positive: {positive_sentiment:.2f}%")
    print(f"   {get_sentiment_emoji(negative_sentiment)} Negative: {negative_sentiment:.2f}%")
    print(f"   {get_sentiment_emoji(neutral_sentiment)} Neutral:  {neutral_sentiment:.2f}%")

    print(f"\n{'-' * 80}")

🤖 Sentiment Analysis

📌 Sentence 1

📜 Original:
   Python is a high-level, general-purpose programming language. Its design
   philosophy emphasizes code readability with the use of significant
   indentation.[33]

📊 Sentiment Analysis Statistics:
   😔 Positive: 9.60%
   😔 Negative: 0.00%
   😃 Neutral:  90.40%

--------------------------------------------------------------------------------
🤖 Sentiment Analysis

📌 Sentence 2

📜 Original:
   Python is dynamically typed and garbage-collected. It supports multiple
   programming paradigms, including structured (particularly procedural),
   object-oriented and functional programming. It is often described as a
   "batteries included" language due to its comprehensive standard
   library.[34][35]

📊 Sentiment Analysis Statistics:
   😔 Positive: 18.90%
   😔 Negative: 0.00%
   😃 Neutral:  81.10%

--------------------------------------------------------------------------------
🤖 Sentiment Analysis

📌 Sentence 3

📜 Original:
   Guido van Rossum